## Import Libraries

In [ ]:
import pandas as pd
import numpy as np

## Loading the Data

In [ ]:
#loan_data_inputs_train = pd.read_csv('loan_data_inputs_train.csv', index_col = 0)
#loan_data_targets_train = pd.read_csv('loan_data_targets_train.csv', index_col = 0, header =  None)
#loan_data_inputs_test = pd.read_csv('loan_data_inputs_test.csv', index_col = 0)
#loan_data_targets_test = pd.read_csv('loan_data_targets_test.csv', index_col = 0, header = None)

In [ ]:
loan_data_backup = pd.read_csv('loan_data_2015.csv')

In [ ]:
df = loan_data_backup.copy()

In [ ]:
df.shape

In [ ]:
pd.options.display.max_columns = None

In [ ]:
#loan_data.head()

In [ ]:
#loan_data.info()

## Population Stability Index: Preprocessing

### General Preprocessing

In [ ]:
df['emp_length'].unique()

We want to change the above values into integers instead of strings. 

In [ ]:
df['emp_length_int'] = df['emp_length'].str.replace('\+ years', '')
df['emp_length_int'] = df['emp_length_int'].str.replace('< 1 year', str(0))
df['emp_length_int'] = df['emp_length_int'].str.replace('n/a',  str(0))
df['emp_length_int'] = df['emp_length_int'].str.replace(' years', '')
df['emp_length_int'] = df['emp_length_int'].str.replace(' year', '')

In [ ]:
type(df['emp_length_int'][0])

In [ ]:
df['emp_length_int'].unique()

In [ ]:
df['emp_length_int'] = pd.to_numeric(df['emp_length_int'])

In [ ]:
type(df['emp_length_int'][0])

In [ ]:
df['earliest_cr_line']

Let's turn these strings into timestamps

In [ ]:
df['earliest_cr_line_date'] = pd.to_datetime(df['earliest_cr_line'], format = '%b-%y')

Here, the %b is to look for the first 3 letters of a month into a string.

%y is to look for the last two digits of the year.

In [ ]:
df['earliest_cr_line_date']

In [ ]:
type(df['earliest_cr_line_date'][0])

In [ ]:
type(df['earliest_cr_line'][0])

In [ ]:
pd.to_datetime('2017-12-01') - df['earliest_cr_line_date']

Let's work with months instead of days

In [ ]:
df['mths_since_earliest_cr_line'] = round(pd.to_numeric((pd.to_datetime('2017-12-01') - df['earliest_cr_line_date']) / np.timedelta64(1,'M')))

In [ ]:
df['mths_since_earliest_cr_line'].describe()

In [ ]:
df.loc[: , ['earliest_cr_line', 'earliest_cr_line_date', 'mths_since_earliest_cr_line']][df['mths_since_earliest_cr_line'] < 0]


In [ ]:
df['mths_since_earliest_cr_line'][df['mths_since_earliest_cr_line']<0] = df['mths_since_earliest_cr_line'].max()

In [ ]:
min(df['mths_since_earliest_cr_line'])

Let's preprocess the 'term' variable as well

In [ ]:
df['term'].describe()

In [ ]:
df['term_int'] = df['term'].str.replace(' months', '')

In [ ]:
df['term_int']

In [ ]:
df['term_int'] = pd.to_numeric(df['term'].str.replace(' months', ''))

In [ ]:
df['term_int'].unique()

Let's look at two other variables: Date of earliest credit line, and loan issued date.

The former is string variables but they are supposed to be date variables.

Now let's preprocess the issue date variable

In [ ]:
df['issue_d']

In [ ]:
df['issue_date'] = pd.to_datetime(df['issue_d'], format = '%b-%y')

In [ ]:
df['mths_since_issue_d'] = round(pd.to_numeric((pd.to_datetime('2017-12-01') - df['issue_date']) / np.timedelta64(1, 'M')))

In [ ]:
df['issue_date']

In [ ]:
type(df['issue_date'][0])

### Preprocessing Discrete Variables

In [ ]:
df.info()

In [ ]:
pd.get_dummies(df['grade'])

We want these names to be a bit more explicit.

In [ ]:
pd.get_dummies(df['grade'], prefix='grade', prefix_sep=":")

We can preprocess the other categorical variables

In [ ]:
df_dummies = [pd.get_dummies(df['grade'], prefix = 'grade', prefix_sep = ':'),
                     pd.get_dummies(df['sub_grade'], prefix = 'sub_grade', prefix_sep = ':'),
                     pd.get_dummies(df['home_ownership'], prefix = 'home_ownership', prefix_sep = ':'),
                     pd.get_dummies(df['verification_status'], prefix = 'verification_status', prefix_sep = ':'),
                     pd.get_dummies(df['loan_status'], prefix = 'loan_status', prefix_sep = ':'),
                     pd.get_dummies(df['purpose'], prefix = 'purpose', prefix_sep = ':'),
                     pd.get_dummies(df['addr_state'], prefix = 'addr_state', prefix_sep = ':'),
                     pd.get_dummies(df['initial_list_status'], prefix = 'initial_list_status', prefix_sep = ':')]


Now let's turn this into a dataframe.

In [ ]:
df_dummies = pd.concat(df_dummies, axis = 1)

In [ ]:
type(df_dummies)

Now we need to concatonate this to the original loans df dataframe.

In [ ]:
df = pd.concat([df, df_dummies], axis = 1)
#axis=1 is to concatonate by columns

In [ ]:
df.columns.values

### Checking for missing values and cleaning

In [ ]:
df.isnull()

Let's change these to binary values. But first lets change the display options to show all rows.

In [ ]:
pd.options.display.max_rows = None
df.isnull().sum()

Let's impute these values.

In [ ]:
pd.options.display.max_rows = 100

In [ ]:
df['total_rev_hi_lim'].fillna(df['funded_amnt'], inplace=True)

In [ ]:
df['total_rev_hi_lim'].isnull().sum()

Let's impute the rest of these values.

We'll replace the nan values for annaul_inc with the mean.

The rest of the variables will have their nan values replaced with zero.

In [ ]:
df['annual_inc'].fillna(df['annual_inc'].mean(), inplace=True)

In [ ]:
pd.options.display.max_rows = None
df.isnull().sum()

In [ ]:
df['mths_since_earliest_cr_line'].fillna(0, inplace=True)
df['acc_now_delinq'].fillna(0, inplace=True)
df['total_acc'].fillna(0, inplace=True)
df['pub_rec'].fillna(0, inplace=True)
df['open_acc'].fillna(0, inplace=True)
df['inq_last_6mths'].fillna(0, inplace=True)
df['delinq_2yrs'].fillna(0, inplace=True)
df['emp_length_int'].fillna(0, inplace=True)

## PD Model

### Data Preparation

#### Dependent Vriable. Good/Bad Definition. Defaullt and Non-Default Accounts

In [ ]:
df['loan_status'].unique()

Let's see how many accounts there are for each status

In [ ]:
df['loan_status'].value_counts()

In [ ]:
df['loan_status'].value_counts()/df['loan_status'].count()

Here, we'll store the non-default, default values to a new variable.

0 will be non-default (good) and 1 will be default (bad).

In [ ]:
# Good/ Bad Definition
df['good_bad'] = np.where(df['loan_status'].isin(['Charged Off', 'Default',
                                                  'Does not meet the credit policy. Status:Charged Off',
                                                  'Late (31-120 days)']), 0, 1)

Here, the where function is checking if the given condition is true.

So, if df['loan_status'] is in either of the three values passed, then pass 0, else pass a 1.

In [ ]:
df['good_bad']

In [ ]:
df.to_csv('2015_prep.csv')


Side note. What is the Weight of Evidence? It is calculated as:

ln((%good)/(%bad))

This calculate to what extent and indpt variable would predict a dep variable.

The Information Value mesaures how much info the original independent variable brings with respect to explaining the dependent variable. The range is always btwn 0 and 1. Here is how to interpret the values.

0 - 1: Predictive Powers

< 0.02: No Predicive Power

.02 < x < .1: Weak Predictive Power

.1 < x < .3: Medium Predictive Power

.3 < x < .5: Strong Predictive Power

.5 < : Suspiciously strong predictive power, too good to be true.

In the example screenshot:

.1 comes from (4000/16000) - (600/4000) = .1

-.1 comes from (12000/16000) - (3400/4000) = -.1

### Splitting the Data - Not really needed for this noteboook

In [ ]:
#from sklearn.model_selection import train_test_split

In [ ]:
#train_test_split(df.drop('good_bad', axis=1), df['good_bad'])

In [ ]:
#loan_data_inputs_train, loan_data_inputs_test, loan_data_targets_train, loan_data_targets_test = train_test_split(df.drop('good_bad', axis = 1), df['good_bad'], test_size = 0.2, random_state = 42)


In [ ]:
#loan_data_inputs_train.shape #x_train

In [ ]:
#loan_data_targets_train.shape #y_train

In [ ]:
#loan_data_inputs_test.shape #x_test

In [ ]:
#loan_data_targets_test.shape #y_test

### Data Prepration Example

* Discrete variables do not need fine classing (splitting into cetegories) b/c they are already categorical by definition.

In [ ]:
df_inputs = df.drop('good_bad', axis=1)
df_targets = df['good_bad']
#df_inputs = loan_data_inputs_test
#df_targets = loan_data_targets_test

In [ ]:
df_inputs['grade'].unique()

Let's create a new dataframe where we'll store only the indpt variable grade from the df_inputs and the dep variable good_bad from the df_targets.

In [ ]:
df1 = pd.concat([df_inputs['grade'], df_targets], axis=1)
df1.head()

Here, A has the highest credit worthiness, while D is the lowest credit worthiness.

Now we'll calculate the weight of evidence this discrete variable. To do this for grade, we must first find the proportion of good and bad borrowers by grade. We'll start with the # of borrowers by each grade.  We can count the rows that contain each of the grades.

We do this using the group by method. We want this code to be reusable, so let's parameterize it.

Grade is the first col in df1, so we we pass values[0].

We set as_index=False b/c we having them as indexes is limiting. It's better to have them as ordinary values.

We then refer to the second col in our df using values[1]

In [ ]:
df1.groupby(df1.columns.values[0], as_index = False)[df1.columns.values[1]].count()


The result contains all grades in the first col, and the # of rows for each grade.

Now we'll calculate the proportion of good borrowers. We can simply get this by calculating the average of good_bad (b/c good=0 and bad=1)

In [ ]:
df1.groupby(df1.columns.values[0], as_index=False)[df1.columns.values[1]].mean()

Now let's merge these two results into one dataframe.

In [ ]:
df1 = pd.concat([df1.groupby(df1.columns.values[0], as_index = False)[df1.columns.values[1]].count(),
                 df1.groupby(df1.columns.values[0], as_index = False)[df1.columns.values[1]].mean()], axis = 1)


In [ ]:
df1

We can see the results match b/c of the duplicate grades col. Let's drop the second one.

In [ ]:
df1 = df1.iloc[:, [0, 1, 3]]
df1

In [ ]:
df1.columns = [df1.columns.values[0], 'n_obs', 'prop_good']
df1

Let's calculate the proportion of observations that fall under each grade.

In [ ]:
df1['prob_n_obs'] = df1['n_obs'] / df1['n_obs'].sum()

In [ ]:
df1

First we'll calculate the number of good borrowers and the number of bad borrowers by the grade group.

We'll store the number of good borrowers as a new variable equal to the proportion of good borrowers multiplied by the total amount of observations that have the respective grade. 

In [ ]:
df1['n_good'] = df1['prop_good'] * df1['n_obs']
df1['n_bad'] = (1 - df1['prop_good']) * df1['n_obs']
df1

Now we'll calculate the proportion of good borrowers to bad borrowers for each grade. 

In [ ]:
df1['prop_n_good'] = df1['n_good'] / df1['n_good'].sum()
df1['prop_n_bad'] = df1['n_bad'] / df1['n_bad'].sum()
df1

Now we have everything we need to calculate the Weight of Evidence for the categories of Grade.

In [ ]:
df1['WoE'] = np.log(df1['prop_n_good'] / df1['prop_n_bad'])
df1

Its useful to sort the result by WoE starting with the category with the lowest weight .

In this way, we'll see the borrowers with the highest default rate first.

In [ ]:
df1 = df1.sort_values(['WoE'])
df1 = df1.reset_index(drop = True)
df1

Let's calculate the proportion of good loans btwn two subsequent categories and the difference of weight in evidence btwn two subsequent categories.

In [ ]:
df1['diff_prop_good'] = df1['prop_good'].diff().abs()
df1['diff_WoE'] = df1['WoE'].diff().abs()
df1

Now we can finally calculate the Information Value.

In [ ]:
df1['IV'] = (df1['prop_n_good'] - df1['prop_n_bad']) * df1['WoE']
df1['IV'] = df1['IV'].sum()
df1

### Preprocessing Discrete Variables: Automating Calculations

In [ ]:
# WoE function for discrete unordered variables
def woe_discrete(df, discrete_variabe_name, good_bad_variable_df):
    df = pd.concat([df[discrete_variabe_name], good_bad_variable_df], axis = 1)
    df = pd.concat([df.groupby(df.columns.values[0], as_index = False)[df.columns.values[1]].count(),
                    df.groupby(df.columns.values[0], as_index = False)[df.columns.values[1]].mean()], axis = 1)
    df = df.iloc[:, [0, 1, 3]]
    df.columns = [df.columns.values[0], 'n_obs', 'prop_good']
    df['prop_n_obs'] = df['n_obs'] / df['n_obs'].sum()
    df['n_good'] = df['prop_good'] * df['n_obs']
    df['n_bad'] = (1 - df['prop_good']) * df['n_obs']
    df['prop_n_good'] = df['n_good'] / df['n_good'].sum()
    df['prop_n_bad'] = df['n_bad'] / df['n_bad'].sum()
    df['WoE'] = np.log(df['prop_n_good'] / df['prop_n_bad'])
    df = df.sort_values(['WoE'])
    df = df.reset_index(drop = True)
    df['diff_prop_good'] = df['prop_good'].diff().abs()
    df['diff_WoE'] = df['WoE'].diff().abs()
    df['IV'] = (df['prop_n_good'] - df['prop_n_bad']) * df['WoE']
    df['IV'] = df['IV'].sum()
    return df

In [ ]:
df_temp = woe_discrete(df_inputs, 'grade', df_targets)
df_temp

It works! This is identical to all the work we had done previously.

### Preprocessing Discrete Variables: Visualizing Results

we want to visualize:

x-axis: the categories of the indpnt variable

y-axis: the weight of evidence

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
def plot_by_woe(df_woe, rotation_of_x_axis_labels=0):
    x = np.array(df_woe.iloc[: , 0].apply(str)) #turn content of 1st col into strings 
    y = df_woe['WoE']
    plt.figure(figsize=(18, 6))
    plt.plot(x, y, marker = 'o', linestyle = '--', color = 'k')
    plt.xlabel(df_woe.columns[0])
    plt.ylabel("Weight of Evidence")
    plt.title(str("Weight of Evidence by " + df_woe.columns[0]))
    plt.xticks(rotation = rotation_of_x_axis_labels)

In [ ]:
plot_by_woe(df_temp)

### Preprocessing Discrete Variables: Creating Dummy Variables Pt. 1

In [ ]:
df_temp = woe_discrete(df_inputs, 'home_ownership', df_targets)
df_temp

In [ ]:
plot_by_woe(df_temp)

It is worth noting that Other and None have the highest prob of default.

Let's combine underrepresented variables that are similar to them (Rent, Other, None, Any).

In [ ]:
df_inputs['home_ownership:Rent_Other_None_Any'] = sum([df_inputs['home_ownership:RENT'],
                                                       df_inputs['home_ownership:ANY']])
#Other and None have been removed                                                      

In [ ]:
df['home_ownership'].unique() #Notice that 'Other' and 'None' are missing.

### Preprocessing Discrete Variables: Creating Dummies Pt. 2

In [ ]:
df_inputs['addr_state'].unique()

In [ ]:
df_temp = woe_discrete(df_inputs, 'addr_state', df_targets)
df_temp

The two missing states are ID and IA.

In [ ]:
plot_by_woe(df_temp)

In [ ]:
if ['addr_state:ND'] in df_inputs.columns.values:
    pass
else:
    df_inputs['addr_state:ND'] = 0

In [ ]:
if ['addr_state:ID'] in df_inputs.columns.values:
    pass
else:
    df_inputs['addr_state:ID'] = 0

In [ ]:
if ['addr_state:IA'] in df_inputs.columns.values:
    pass
else:
    df_inputs['addr_state:IA'] = 0

In [ ]:
plot_by_woe(df_temp.iloc[2: -2, :]) #from row 2 to the third to last row

This new graph gives us a different perspective. The previous graph was telling us that most states are somewhat within the same range of WoE.

However, we can clearly see there are some differences. It may make sense to keep NV and HI in a separate category, and keep the next 6 states in another category.

We're going to combine the first 6 states into one category, and the last 6 states as well. So let's look at the remaining states.

In [ ]:
plot_by_woe(df_temp.iloc[6: -6, :])

This result is more or less what we expect from a typical WoE plot. Not let'se  try to create the categories.

These will be our categories for our dummies. We judged these categories based on their WoE and the amount of observations each state had.

We'll leave assigned one dummy variable as a reference category in the regression model. This is going to be the variable with the lowest WoE (So from IA to FL).

Remember, we already have dummy variables for all the states. So for the categories with only one state, we can use these dummy variables and not have to create new ones.

For the categories with 2 or more states, we can do a similar process to what we did for the categories of home ownership. We will sum the dummy variables for every state that is part of a bigger category. This will result in a dummy variable for the whole category. 

In [ ]:
df_inputs['addr_state:ND_NE_IA_NV_FL_HI_AL'] = sum([df_inputs['addr_state:ND'], df_inputs['addr_state:NE'],
                                              df_inputs['addr_state:IA'], df_inputs['addr_state:NV'],
                                              df_inputs['addr_state:FL'], df_inputs['addr_state:HI'],
                                                          df_inputs['addr_state:AL']])

df_inputs['addr_state:NM_VA'] = sum([df_inputs['addr_state:NM'], df_inputs['addr_state:VA']])

df_inputs['addr_state:OK_TN_MO_LA_MD_NC'] = sum([df_inputs['addr_state:OK'], df_inputs['addr_state:TN'],
                                              df_inputs['addr_state:MO'], df_inputs['addr_state:LA'],
                                              df_inputs['addr_state:MD'], df_inputs['addr_state:NC']])

df_inputs['addr_state:UT_KY_AZ_NJ'] = sum([df_inputs['addr_state:UT'], df_inputs['addr_state:KY'],
                                              df_inputs['addr_state:AZ'], df_inputs['addr_state:NJ']])

df_inputs['addr_state:AR_MI_PA_OH_MN'] = sum([df_inputs['addr_state:AR'], df_inputs['addr_state:MI'],
                                              df_inputs['addr_state:PA'], df_inputs['addr_state:OH'],
                                              df_inputs['addr_state:MN']])

df_inputs['addr_state:RI_MA_DE_SD_IN'] = sum([df_inputs['addr_state:RI'], df_inputs['addr_state:MA'],
                                              df_inputs['addr_state:DE'], df_inputs['addr_state:SD'],
                                              df_inputs['addr_state:IN']])

df_inputs['addr_state:GA_WA_OR'] = sum([df_inputs['addr_state:GA'], df_inputs['addr_state:WA'],
                                              df_inputs['addr_state:OR']])

df_inputs['addr_state:WI_MT'] = sum([df_inputs['addr_state:WI'], df_inputs['addr_state:MT']])

df_inputs['addr_state:IL_CT'] = sum([df_inputs['addr_state:IL'], df_inputs['addr_state:CT']])

df_inputs['addr_state:KS_SC_CO_VT_AK_MS'] = sum([df_inputs['addr_state:KS'], df_inputs['addr_state:SC'],
                                              df_inputs['addr_state:CO'], df_inputs['addr_state:VT'],
                                              df_inputs['addr_state:AK'], df_inputs['addr_state:MS']])

df_inputs['addr_state:WV_NH_WY_DC_ME_ID'] = sum([df_inputs['addr_state:WV'], df_inputs['addr_state:NH'],
                                              df_inputs['addr_state:WY'], df_inputs['addr_state:DC'],
                                              df_inputs['addr_state:ME'], df_inputs['addr_state:ID']])

There's still 3 discrete variables left for preprocessing: verification_status, purpose, and initial_list_status.

Let's examin the WoE for these 3 variables and determine the categories for each of them as dummy variables for the probability of default (PD) model.

In [ ]:
df_temp = woe_discrete(df_inputs, 'verification_status', df_targets)
df_temp

In [ ]:
plot_by_woe(df_temp)

In [ ]:
#Dummy Variables not needed

In [ ]:
df_temp = woe_discrete(df_inputs, 'purpose', df_targets)
df_temp

In [ ]:
plot_by_woe(df_temp, 90)

In [ ]:
df_inputs['purpose:educ__sm_b__wedd__ren_en__mov__house'] = sum([df_inputs['purpose:educational'], df_inputs['purpose:small_business'],
                                                                 df_inputs['purpose:wedding'], df_inputs['purpose:renewable_energy'],
                                                                 df_inputs['purpose:moving'], df_inputs['purpose:house']])

df_inputs['purpose:oth__med__vacation'] = sum([df_inputs['purpose:other'], df_inputs['purpose:medical'],
                                               df_inputs['purpose:vacation']])

df_inputs['purpose:major_purch__car__home_impr'] = sum([df_inputs['purpose:major_purchase'], df_inputs['purpose:car'],
                                                        df_inputs['purpose:home_improvement']])


In [ ]:
df_temp = woe_discrete(df_inputs, 'initial_list_status', df_targets)
df_temp

In [ ]:
plot_by_woe(df_temp)

In [ ]:
#^No Dummies Needed

### Preprocessing Continuous Variables: Automating Calculations and Visualizations

In [ ]:
# WoE function for ordered discrete and continuous variables
def woe_ordered_cont(df, discrete_variabe_name, good_bad_variable_df):
    df = pd.concat([df[discrete_variabe_name], good_bad_variable_df], axis = 1)
    df = pd.concat([df.groupby(df.columns.values[0], as_index = False)[df.columns.values[1]].count(),
                    df.groupby(df.columns.values[0], as_index = False)[df.columns.values[1]].mean()], axis = 1)
    df = df.iloc[:, [0, 1, 3]]
    df.columns = [df.columns.values[0], 'n_obs', 'prop_good']
    df['prop_n_obs'] = df['n_obs'] / df['n_obs'].sum()
    df['n_good'] = df['prop_good'] * df['n_obs']
    df['n_bad'] = (1 - df['prop_good']) * df['n_obs']
    df['prop_n_good'] = df['n_good'] / df['n_good'].sum()
    df['prop_n_bad'] = df['n_bad'] / df['n_bad'].sum()
    df['WoE'] = np.log(df['prop_n_good'] / df['prop_n_bad'])
    #df = df.sort_values(['WoE'])
    #df = df.reset_index(drop = True)
    df['diff_prop_good'] = df['prop_good'].diff().abs()
    df['diff_WoE'] = df['WoE'].diff().abs()
    df['IV'] = (df['prop_n_good'] - df['prop_n_bad']) * df['WoE']
    df['IV'] = df['IV'].sum()
    return df
# Here we define a function similar to the one above, ...
# ... with one slight di

We don't want the categories to be ordered by WoE, we want the natural order!

### Preprocessing Continuous Variables: Creating Dummy Variables Pt 1

In [ ]:
df_inputs['term_int'].unique()

In [ ]:
df_temp = woe_ordered_cont(df_inputs, 'term_int', df_targets)
df_temp

In [ ]:
plot_by_woe(df_temp)

Seems like 60 month loans are much riskier than 36 month loans. Let's create the dummy variables

In [ ]:
df_inputs['term:36'] = np.where((df_inputs['term_int'] == 36), 1, 0)
df_inputs['term:60'] = np.where((df_inputs['term_int'] == 60), 1, 0)


In [ ]:
df_inputs['emp_length_int'].unique()

In [ ]:
df_temp = woe_ordered_cont(df_inputs, 'emp_length_int', df_targets)
df_temp

In [ ]:
plot_by_woe(df_temp)

Now we split up by categories. (Note, 10 will be in its own category). Borrowers with 0 years of employment are riskiest.

In [ ]:
df_inputs['emp_length:0'] = np.where(df_inputs['emp_length_int'].isin([0]), 1, 0)
df_inputs['emp_length:1'] = np.where(df_inputs['emp_length_int'].isin([1]), 1, 0)
df_inputs['emp_length:2-4'] = np.where(df_inputs['emp_length_int'].isin(range(2, 5)), 1, 0)
df_inputs['emp_length:5-6'] = np.where(df_inputs['emp_length_int'].isin(range(5, 7)), 1, 0)
df_inputs['emp_length:7-9'] = np.where(df_inputs['emp_length_int'].isin(range(8, 10)), 1, 0)
df_inputs['emp_length:10'] = np.where(df_inputs['emp_length_int'].isin([10]), 1, 0)

^Here, we're checking if the employment length int variable is in a certain list of values.

### Preprocessing Continuous Variables: Creating Dummy Variables Pt 2

In [ ]:
df_inputs['mths_since_issue_d'].unique()

Let's quickly define fine and coarse classing.

Fine Classing - Roughly groupping the values into categories

Coarse Classing: Determining final categories, combining few of the initial fine categories into bigger categories if needed.

Suppose we wanted to split the category into say, 50 categoeries. We would use pandas cut.

*Note* Factor = Categorical variable

In [ ]:
df_inputs['mths_since_issue_d_factor'] = pd.cut(df_inputs['mths_since_issue_d'], 50)

In [ ]:
df_inputs['mths_since_issue_d_factor']

When we scroll to the bottom of this cell, we see a summary of the corresponding to the lowest values of the original continuous variable and the highest values. Now we can treat this new cat var just like any other cat variable so far.

In [ ]:
df_temp = woe_ordered_cont(df_inputs, 'mths_since_issue_d_factor', df_targets)
df_temp

We get a df that contains the results with the same calculations as for any other variable we've explored so far. The only difference is the first column, where our indpt variable is contained. We can see the 50 intervals of the original indpt variable.

In [ ]:
plot_by_woe(df_temp, 90)

We can now start coarse classing the variable.

The 1st, 2nd, and 3rd categories differ significantly in terms of WoE! Let's keep each of them as categories for the final model. Remember the mths_since var takes integer values only.

In [ ]:
plot_by_woe(df_temp.iloc[3: , :], 90)

No we see the first 2 values are not too different, so let's have that as a category.

Now, since the next intervals only refer to a small portion of our data, we can split the ret of the variables into just two categories (down the middle, or at 84 months since issue date). 

In [ ]:
df_inputs['mths_since_issue_d:<38'] = np.where(df_inputs['mths_since_issue_d'].isin(range(38)), 1, 0)
df_inputs['mths_since_issue_d:38-39'] = np.where(df_inputs['mths_since_issue_d'].isin(range(38,40)), 1, 0)
df_inputs['mths_since_issue_d:40-41'] = np.where(df_inputs['mths_since_issue_d'].isin(range(40,42)), 1, 0)
df_inputs['mths_since_issue_d:42-48'] = np.where(df_inputs['mths_since_issue_d'].isin(range(42, 49)), 1, 0)
df_inputs['mths_since_issue_d:49-52'] = np.where(df_inputs['mths_since_issue_d'].isin(range(49, 53)), 1, 0)
df_inputs['mths_since_issue_d:53-64'] = np.where(df_inputs['mths_since_issue_d'].isin(range(53, 65)), 1, 0)
df_inputs['mths_since_issue_d:65-84'] = np.where(df_inputs['mths_since_issue_d'].isin(range(65, 85)), 1, 0)
df_inputs['mths_since_issue_d:>84'] = np.where(df_inputs['mths_since_issue_d'].isin(range(85, int(df_inputs['mths_since_issue_d'].max()))), 1, 0)

Let's do another variable.

In [ ]:
df_inputs['int_rate_factor'] = pd.cut(df_inputs['int_rate'], 50)

In [ ]:
df_temp = woe_ordered_cont(df_inputs, 'int_rate_factor', df_targets)
df_temp

In [ ]:
plot_by_woe(df_temp, 90)

As we can see, the relationship btwn interest rate and WoE is almost monotonic. The greater the interest rate, the lower the WoE, and the higher probability of defualt. Let's bundle them up.

The numerous ups and downs in the first 10 points remind us to check the number of observations for each point.  WoE is higher for them, but there's not many observations (in relation to the rest of the data), we we'll bundle them in one dummy.

In [ ]:
df_inputs['int_rate:<9.548'] = np.where((df_inputs['int_rate'] <= 9.548), 1, 0)
df_inputs['int_rate:9.548-12.025'] = np.where((df_inputs['int_rate'] > 9.548) & (df_inputs['int_rate'] <= 12.025), 1, 0)
df_inputs['int_rate:12.025-15.74'] = np.where((df_inputs['int_rate'] > 12.025) & (df_inputs['int_rate'] <= 15.74), 1, 0)
df_inputs['int_rate:15.74-20.281'] = np.where((df_inputs['int_rate'] > 15.74) & (df_inputs['int_rate'] <= 20.281), 1, 0)
df_inputs['int_rate:>20.281'] = np.where((df_inputs['int_rate'] > 20.281), 1, 0)

In [ ]:
df_inputs['funded_amnt_factor'] = pd.cut(df_inputs['funded_amnt'], 50)
df_temp = woe_ordered_cont(df_inputs, 'funded_amnt_factor', df_targets)
df_temp

In [ ]:
plot_by_woe(df_temp, 90)

We see a trend that almost resembles a horizontal line. There is no other clear pattern of dependence btwn the indpt variable and WoE. In other words, WoE varies greatly and there seems to be no association with the indpt variable. This is sufficient evidence to *NOT* use the 'funded_amnt' variable in our PD model. So there will be no need to create dummy variables. 

Let's do the same process for the following variables:

· ‘mths_since_earliest_cr_line’;

· ‘installment’;

· ‘delinq_2yrs’;

· ‘inq_last_6mths’;

· ‘open_acc’;

· ‘pub_rec’;

· ‘total_acc’;

· ‘acc_now_delinq’

In [ ]:
df_inputs['mths_since_earliest_cr_line'].unique()

In [ ]:
df_inputs['mths_since_earliest_cr_line_factor'] = pd.cut(df_inputs['mths_since_earliest_cr_line'], 50)

In [ ]:
df_inputs['mths_since_earliest_cr_line_factor']

In [ ]:
df_temp = woe_ordered_cont(df_inputs, 'mths_since_earliest_cr_line_factor', df_targets)
df_temp

In [ ]:
plot_by_woe(df_temp, 90)

In [ ]:
plot_by_woe(df_temp.iloc[6: , : ], 90)

In [ ]:

df_inputs['mths_since_earliest_cr_line:<140'] = np.where(df_inputs['mths_since_earliest_cr_line'].isin(range(140)), 1, 0)
df_inputs['mths_since_earliest_cr_line:141-164'] = np.where(df_inputs['mths_since_earliest_cr_line'].isin(range(140, 165)), 1, 0)
df_inputs['mths_since_earliest_cr_line:165-247'] = np.where(df_inputs['mths_since_earliest_cr_line'].isin(range(165, 248)), 1, 0)
df_inputs['mths_since_earliest_cr_line:248-270'] = np.where(df_inputs['mths_since_earliest_cr_line'].isin(range(248, 271)), 1, 0)
df_inputs['mths_since_earliest_cr_line:271-352'] = np.where(df_inputs['mths_since_earliest_cr_line'].isin(range(271, 353)), 1, 0)
df_inputs['mths_since_earliest_cr_line:>352'] = np.where(df_inputs['mths_since_earliest_cr_line'].isin(range(353, int(df_inputs['mths_since_earliest_cr_line'].max()))), 1, 0)


We will preprocess the 'installment' variable similarly to how we preprocessed 'funded_amnt'

We will now preprocess the remaining variables similarly to the 'int_rate' variable.

Now we will look at 'delinq_2yrs'

In [ ]:
df_inputs['delinq_2yrs'].unique()

In [ ]:
df_temp = woe_ordered_cont(df_inputs, 'delinq_2yrs', df_targets)
df_temp

In [ ]:
plot_by_woe(df_temp, 90)

In [ ]:
df_inputs['delinq_2yrs:0'] = np.where((df_inputs['delinq_2yrs'] == 0), 1, 0)
df_inputs['delinq_2yrs:1-3'] = np.where((df_inputs['delinq_2yrs'] >= 1) & (df_inputs['delinq_2yrs'] <= 3), 1, 0)
df_inputs['delinq_2yrs:>=4'] = np.where((df_inputs['delinq_2yrs'] >= 9), 1, 0)


Now we'll look at inq_last_6mths

In [ ]:
df_inputs['inq_last_6mths'].unique()

In [ ]:
df_temp = woe_ordered_cont(df_inputs, 'inq_last_6mths', df_targets)
df_temp

In [ ]:
plot_by_woe(df_temp, 90)

In [ ]:
df_inputs['inq_last_6mths:0'] = np.where((df_inputs['inq_last_6mths'] == 0), 1, 0)
df_inputs['inq_last_6mths:1-2'] = np.where((df_inputs['inq_last_6mths'] >= 1) & (df_inputs['inq_last_6mths'] <= 2), 1, 0)
df_inputs['inq_last_6mths:3-6'] = np.where((df_inputs['inq_last_6mths'] >= 3) & (df_inputs['inq_last_6mths'] <= 6), 1, 0)
df_inputs['inq_last_6mths:>6'] = np.where((df_inputs['inq_last_6mths'] > 6), 1, 0)


In [ ]:
df_inputs['open_acc'].unique()

In [ ]:
# open_acc
df_temp = woe_ordered_cont(df_inputs, 'open_acc', df_targets)
# We calculate weight of evidence.
df_temp

In [ ]:
plot_by_woe(df_temp.iloc[ : 40, :], 90)

Starting at point 20, the number of observations seem to dwindle, so we can group all these values into one dummy variable.

In [ ]:
# Categories: '0', '1-3', '4-12', '13-17', '18-22', '23-25', '26-30', '>30'
df_inputs['open_acc:0'] = np.where((df_inputs['open_acc'] == 0), 1, 0)
df_inputs['open_acc:1-3'] = np.where((df_inputs['open_acc'] >= 1) & (df_inputs['open_acc'] <= 3), 1, 0)
df_inputs['open_acc:4-12'] = np.where((df_inputs['open_acc'] >= 4) & (df_inputs['open_acc'] <= 12), 1, 0)
df_inputs['open_acc:13-17'] = np.where((df_inputs['open_acc'] >= 13) & (df_inputs['open_acc'] <= 17), 1, 0)
df_inputs['open_acc:18-22'] = np.where((df_inputs['open_acc'] >= 18) & (df_inputs['open_acc'] <= 22), 1, 0)
df_inputs['open_acc:23-25'] = np.where((df_inputs['open_acc'] >= 23) & (df_inputs['open_acc'] <= 25), 1, 0)
df_inputs['open_acc:26-30'] = np.where((df_inputs['open_acc'] >= 26) & (df_inputs['open_acc'] <= 30), 1, 0)
df_inputs['open_acc:>=31'] = np.where((df_inputs['open_acc'] >= 31), 1, 0)


Now for pub_rec

In [ ]:
# pub_rec
df_temp = woe_ordered_cont(df_inputs, 'pub_rec', df_targets)
# We calculate weight of evidence.
df_temp

In [ ]:
plot_by_woe(df_temp, 90)

In [ ]:
# Categories '0-2', '3-4', '>=5'
df_inputs['pub_rec:0-2'] = np.where((df_inputs['pub_rec'] >= 0) & (df_inputs['pub_rec'] <= 2), 1, 0)
df_inputs['pub_rec:3-4'] = np.where((df_inputs['pub_rec'] >= 3) & (df_inputs['pub_rec'] <= 4), 1, 0)
df_inputs['pub_rec:>=5'] = np.where((df_inputs['pub_rec'] >= 5), 1, 0)

Now for 'total_acc'

In [ ]:
df_inputs['total_acc'].unique()

In [ ]:
df_inputs['total_acc_factor'] = pd.cut(df_inputs['total_acc'], 50)
df_temp = woe_ordered_cont(df_inputs, 'total_acc_factor', df_targets)
df_temp

In [ ]:
plot_by_woe(df_temp, 90)

In [ ]:
df_inputs['total_acc:<=27'] = np.where((df_inputs['total_acc'] <= 27), 1, 0)
df_inputs['total_acc:28-51'] = np.where((df_inputs['total_acc'] >= 28) & (df_inputs['total_acc'] <= 51), 1, 0)
df_inputs['total_acc:>=52'] = np.where((df_inputs['total_acc'] >= 52), 1, 0)


Now for 'acc_now_delinq'

In [ ]:
df_inputs['acc_now_delinq'].unique()

In [ ]:
#df['acc_now_delinq_factor'] = pd.cut(df_inputs['acc_now_delinq'], 50)
df_temp = woe_ordered_cont(df_inputs, 'acc_now_delinq', df_targets)
df_temp

In [ ]:
plot_by_woe(df_temp)

In [ ]:
df_inputs['acc_now_delinq:0'] = np.where((df_inputs['acc_now_delinq'] == 0), 1, 0)
df_inputs['acc_now_delinq:>=1'] = np.where((df_inputs['acc_now_delinq'] >= 1), 1, 0)

In [ ]:
# total_rev_hi_lim
df_inputs['total_rev_hi_lim_factor'] = pd.cut(df_inputs['total_rev_hi_lim'], 2000)
df_temp = woe_ordered_cont(df_inputs, 'total_rev_hi_lim_factor', df_targets)
df_temp

In [ ]:
plot_by_woe(df_temp.iloc[: 50, : ], 90)

In [ ]:

df_inputs['total_rev_hi_lim:<=5K'] = np.where((df_inputs['total_rev_hi_lim'] <= 5000), 1, 0)
df_inputs['total_rev_hi_lim:5K-10K'] = np.where((df_inputs['total_rev_hi_lim'] > 5000) & (df_inputs['total_rev_hi_lim'] <= 10000), 1, 0)
df_inputs['total_rev_hi_lim:10K-20K'] = np.where((df_inputs['total_rev_hi_lim'] > 10000) & (df_inputs['total_rev_hi_lim'] <= 20000), 1, 0)
df_inputs['total_rev_hi_lim:20K-30K'] = np.where((df_inputs['total_rev_hi_lim'] > 20000) & (df_inputs['total_rev_hi_lim'] <= 30000), 1, 0)
df_inputs['total_rev_hi_lim:30K-40K'] = np.where((df_inputs['total_rev_hi_lim'] > 30000) & (df_inputs['total_rev_hi_lim'] <= 40000), 1, 0)
df_inputs['total_rev_hi_lim:40K-55K'] = np.where((df_inputs['total_rev_hi_lim'] > 40000) & (df_inputs['total_rev_hi_lim'] <= 55000), 1, 0)
df_inputs['total_rev_hi_lim:55K-95K'] = np.where((df_inputs['total_rev_hi_lim'] > 55000) & (df_inputs['total_rev_hi_lim'] <= 95000), 1, 0)
df_inputs['total_rev_hi_lim:>95K'] = np.where((df_inputs['total_rev_hi_lim'] > 95000), 1, 0)


In [ ]:
df_inputs['installment'].unique()

In [ ]:
df_inputs['installment_factor'] = pd.cut(df_inputs['installment'], 50)
df_temp = woe_ordered_cont(df_inputs, 'installment_factor', df_targets)
df_temp

In [ ]:
plot_by_woe(df_temp, 90)

We will not include this in our PD Model.

### Preprocessing Continuous Variables: Creating Dummy Variables Pt. 3

There's still a few variables left that require some special treatment.

Annual_income is the first one. This variable contains values greater than zero, perhaps up to millions.

In [ ]:
df_inputs['annual_inc_factor'] = pd.cut(df_inputs['annual_inc'], 50)
df_temp = woe_ordered_cont(df_inputs, 'annual_inc_factor', df_targets)
df_temp

There's something immediately noticable here. The first interval has a very large amount of num_obs, and contains more than 97% of all observations. Therefore, it seems that 50 categories were not enough to fine class the variable!

What we could do is try to split the variable in more than 50 classes, say 100.

In [ ]:
df_inputs['annual_inc_factor'] = pd.cut(df_inputs['annual_inc'], 100)
df_temp = woe_ordered_cont(df_inputs, 'annual_inc_factor', df_targets)
df_temp

A bit better, but still not that great. Let's try to set aside people with higher income and try to evaluate people with lower income seperately.

Let's create one dummy variable for people with large income. We will assume that $190K is the threshold above which people are considered to have a high income.

In [ ]:
df_inputs_prep_temp = df_inputs.loc[df_inputs['annual_inc'] <= 140000, : ]

In [ ]:
df_inputs_prep_temp['annual_inc_factor'] = pd.cut(df_inputs_prep_temp['annual_inc'], 50)
df_temp = woe_ordered_cont(df_inputs_prep_temp, 'annual_inc_factor', df_targets[df_inputs_prep_temp.index])
df_temp

In [ ]:
plot_by_woe(df_temp, 90)

What we see here is that WoE increases almost monotonically with annual income. Thus, we can safely split the variable into some roughly equal intervals. We need to take into consideration of the number of observations for each interval.

Let's make the first interval <= $20000.

In [ ]:
df_inputs['annual_inc:<20K'] = np.where((df_inputs['annual_inc'] <= 20000), 1, 0)
df_inputs['annual_inc:20K-30K'] = np.where((df_inputs['annual_inc'] > 20000) & (df_inputs['annual_inc'] <= 30000), 1, 0)
df_inputs['annual_inc:30K-40K'] = np.where((df_inputs['annual_inc'] > 30000) & (df_inputs['annual_inc'] <= 40000), 1, 0)
df_inputs['annual_inc:40K-50K'] = np.where((df_inputs['annual_inc'] > 40000) & (df_inputs['annual_inc'] <= 50000), 1, 0)
df_inputs['annual_inc:50K-60K'] = np.where((df_inputs['annual_inc'] > 50000) & (df_inputs['annual_inc'] <= 60000), 1, 0)
df_inputs['annual_inc:60K-70K'] = np.where((df_inputs['annual_inc'] > 60000) & (df_inputs['annual_inc'] <= 70000), 1, 0)
df_inputs['annual_inc:70K-80K'] = np.where((df_inputs['annual_inc'] > 70000) & (df_inputs['annual_inc'] <= 80000), 1, 0)
df_inputs['annual_inc:80K-90K'] = np.where((df_inputs['annual_inc'] > 80000) & (df_inputs['annual_inc'] <= 90000), 1, 0)
df_inputs['annual_inc:90K-100K'] = np.where((df_inputs['annual_inc'] > 90000) & (df_inputs['annual_inc'] <= 100000), 1, 0)
df_inputs['annual_inc:100K-120K'] = np.where((df_inputs['annual_inc'] > 100000) & (df_inputs['annual_inc'] <= 120000), 1, 0)
df_inputs['annual_inc:120K-140K'] = np.where((df_inputs['annual_inc'] > 120000) & (df_inputs['annual_inc'] <= 140000), 1, 0)
df_inputs['annual_inc:>140K'] = np.where((df_inputs['annual_inc'] > 14000), 1, 0)


Let's explore months since last delinquency. There's a lot of missing variables here.

To deal with this, we are going to create a dummy variable indicating that the original variable is missing:

1 = when the value is missing

0 = when the value is not missing.

In [ ]:
# mths_since_last_delinq
df_inputs_prepr_temp = df_inputs[pd.notnull(df_inputs['mths_since_last_delinq'])]
df_inputs_prepr_temp['mths_since_last_delinq_factor'] = pd.cut(df_inputs_prepr_temp['mths_since_last_delinq'], 50)
df_temp = woe_ordered_cont(df_inputs_prepr_temp, 'mths_since_last_delinq_factor', df_targets[df_inputs_prepr_temp.index])
# We calculate weight of evidence.
df_temp

The third line in the cell above, we make df_temp by selecting only the rows with targets that correspond to the rows with inputs we selected, that is: the rows with non missing months since last delinquency.

To do that, we took advantage of the row indexes of the df_inputs_prep_temp, where we have already selected the rows with non missing values.

In [ ]:
plot_by_woe(df_temp, 90)

In [ ]:
# Categories: Missing, 0-3, 4-30, 31-56, >=57
df_inputs['mths_since_last_delinq:Missing'] = np.where((df_inputs['mths_since_last_delinq'].isnull()), 1, 0)
df_inputs['mths_since_last_delinq:0-3'] = np.where((df_inputs['mths_since_last_delinq'] >= 0) & (df_inputs['mths_since_last_delinq'] <= 3), 1, 0)
df_inputs['mths_since_last_delinq:4-30'] = np.where((df_inputs['mths_since_last_delinq'] >= 4) & (df_inputs['mths_since_last_delinq'] <= 30), 1, 0)
df_inputs['mths_since_last_delinq:31-56'] = np.where((df_inputs['mths_since_last_delinq'] >= 31) & (df_inputs['mths_since_last_delinq'] <= 56), 1, 0)
df_inputs['mths_since_last_delinq:>=57'] = np.where((df_inputs['mths_since_last_delinq'] >= 57), 1, 0)


We see that WoE is considerably lower for the first category. Let the first category be from 0 to 3 months.

The following intervals have so few observations that it doesn't make sense to treat them separately. So the last dummy variable will be anything greater than 56 months.

Now we'll preprocess two more continuous varialbes and see if we'll use them for our PD model, which are 'dti’ (debt to income ratio) and 'mths_since_last_record’

In [ ]:
df_inputs['dti'].unique()

In [ ]:
# dti
df_inputs['dti_factor'] = pd.cut(df_inputs['dti'], 100)
# Here we do fine-classing: using the 'cut' method, we split the variable into 100 categories by its values.
df_temp = woe_ordered_cont(df_inputs, 'dti_factor', df_targets)
# We calculate weight of evidence.
df_temp

In [ ]:
plot_by_woe(df_temp, 90)

In [ ]:
# Similarly to income, initial examination shows that most values are lower than 200.
# Hence, we are going to have one category for more than 35, and we are going to apply our approach to determine
# the categories of everyone with 150k or less.
df_inputs_prepr_temp = df_inputs.loc[df_inputs['dti'] <= 35, : ]

In [ ]:
df_inputs_prepr_temp['dti_factor'] = pd.cut(df_inputs['dti'], 50)
# Here we do fine-classing: using the 'cut' method, we split the variable into 50 categories by its values.
df_temp = woe_ordered_cont(df_inputs_prepr_temp, 'dti_factor', df_targets[df_inputs_prepr_temp.index])
# We calculate weight of evidence.
df_temp

In [ ]:
plot_by_woe(df_temp, 90)
# We plot the weight of evidence values.

In [ ]:
# Categories:
df_inputs['dti:<=1.4'] = np.where((df_inputs['dti'] <= 1.4), 1, 0)
df_inputs['dti:1.4-3.5'] = np.where((df_inputs['dti'] > 1.4) & (df_inputs['dti'] <= 3.5), 1, 0)
df_inputs['dti:3.5-7.7'] = np.where((df_inputs['dti'] > 3.5) & (df_inputs['dti'] <= 7.7), 1, 0)
df_inputs['dti:7.7-10.5'] = np.where((df_inputs['dti'] > 7.7) & (df_inputs['dti'] <= 10.5), 1, 0)
df_inputs['dti:10.5-16.1'] = np.where((df_inputs['dti'] > 10.5) & (df_inputs['dti'] <= 16.1), 1, 0)
df_inputs['dti:16.1-20.3'] = np.where((df_inputs['dti'] > 16.1) & (df_inputs['dti'] <= 20.3), 1, 0)
df_inputs['dti:20.3-21.7'] = np.where((df_inputs['dti'] > 20.3) & (df_inputs['dti'] <= 21.7), 1, 0)
df_inputs['dti:21.7-22.4'] = np.where((df_inputs['dti'] > 21.7) & (df_inputs['dti'] <= 22.4), 1, 0)
df_inputs['dti:22.4-35'] = np.where((df_inputs['dti'] > 22.4) & (df_inputs['dti'] <= 35), 1, 0)
df_inputs['dti:>35'] = np.where((df_inputs['dti'] > 35), 1, 0)

In [ ]:
df_inputs['mths_since_last_record'].unique()

In [ ]:
# mths_since_last_record
# We have to create one category for missing values and do fine and coarse classing for the rest.
df_inputs_prepr_temp = df_inputs[pd.notnull(df_inputs['mths_since_last_record'])]
#sum(loan_data_temp['mths_since_last_record'].isnull())

df_inputs_prepr_temp['mths_since_last_record_factor'] = pd.cut(df_inputs_prepr_temp['mths_since_last_record'], 50)
# Here we do fine-classing: using the 'cut' method, we split the variable into 50 categories by its values.

df_temp = woe_ordered_cont(df_inputs_prepr_temp, 'mths_since_last_record_factor', df_targets[df_inputs_prepr_temp.index])
# We calculate weight of evidence.
df_temp

In [ ]:
plot_by_woe(df_temp, 90)
# We plot the weight of evidence values.

In [ ]:
# Categories: 'Missing', '0-2', '3-20', '21-31', '32-80', '81-86', '>86'
df_inputs['mths_since_last_record:Missing'] = np.where((df_inputs['mths_since_last_record'].isnull()), 1, 0)
df_inputs['mths_since_last_record:0-2'] = np.where((df_inputs['mths_since_last_record'] >= 0) & (df_inputs['mths_since_last_record'] <= 2), 1, 0)
df_inputs['mths_since_last_record:3-20'] = np.where((df_inputs['mths_since_last_record'] >= 3) & (df_inputs['mths_since_last_record'] <= 20), 1, 0)
df_inputs['mths_since_last_record:21-31'] = np.where((df_inputs['mths_since_last_record'] >= 21) & (df_inputs['mths_since_last_record'] <= 31), 1, 0)
df_inputs['mths_since_last_record:32-80'] = np.where((df_inputs['mths_since_last_record'] >= 32) & (df_inputs['mths_since_last_record'] <= 80), 1, 0)
df_inputs['mths_since_last_record:81-86'] = np.where((df_inputs['mths_since_last_record'] >= 81) & (df_inputs['mths_since_last_record'] <= 86), 1, 0)
df_inputs['mths_since_last_record:>86'] = np.where((df_inputs['mths_since_last_record'] > 86), 1, 0)


We will not use this for our PD model as well.

### Preprocessing the Test Data

When working with the testing data, we don't need to assess the WoE or come up with categories. Rather, we must create the same dummy variables we used to train the model.

Let's save the preprocessed train data set. We will then scroll all the way back up to the first step of the preprocessing and change the code.

We then go to right up to cell 186 and run all cells before it.

In [ ]:
#loan_data_inputs_train = df_inputs
#loan_data_inputs_test = df_inputs
loan_data_inputs_2015 = df_inputs

In [ ]:
loan_data_targets_2015 = df_targets

In [ ]:
loan_data_inputs_2015.columns.values

In [ ]:
loan_data_inputs_2015.shape

In [ ]:
loan_data_targets_2015.shape

#NOTE

Run all cells above part 3 first, then continue to run all cells below part 3.

In [ ]:
#loan_data_inputs_train.to_csv('loan_data_inputs_train.csv')
#loan_data_targets_train.to_csv('loan_data_targets_train.csv')
#loan_data_inputs_test.to_csv('loan_data_inputs_test.csv')
#loan_data_targets_test.to_csv('loan_data_targets_test.csv')

In [ ]:
loan_data_inputs_2015.to_csv('loan_data_inputs_2015.csv')
loan_data_targets_2015.to_csv('loan_data_targets_2015.csv')

Now let's select the dummies that were actually used in the PD model.

In [ ]:
inputs_train_with_ref_cat = pd.read_csv('inputs_train_with_ref_cat.csv', index_col=0)

In [ ]:
inputs_2015_with_ref_cat = loan_data_inputs_2015[['grade:A',
'grade:B',
'grade:C',
'grade:D',
'grade:E',
'grade:F',
'grade:G',
'home_ownership:Rent_Other_None_Any',
'home_ownership:OWN',
'home_ownership:MORTGAGE',
'addr_state:ND_NE_IA_NV_FL_HI_AL',
'addr_state:NM_VA',
'addr_state:NY',
'addr_state:OK_TN_MO_LA_MD_NC',
'addr_state:CA',
'addr_state:UT_KY_AZ_NJ',
'addr_state:AR_MI_PA_OH_MN',
'addr_state:RI_MA_DE_SD_IN',
'addr_state:GA_WA_OR',
'addr_state:WI_MT',
'addr_state:TX',
'addr_state:IL_CT',
'addr_state:KS_SC_CO_VT_AK_MS',
'addr_state:WV_NH_WY_DC_ME_ID',
'verification_status:Not Verified',
'verification_status:Source Verified',
'verification_status:Verified',
'purpose:educ__sm_b__wedd__ren_en__mov__house',
'purpose:credit_card',
'purpose:debt_consolidation',
'purpose:oth__med__vacation',
'purpose:major_purch__car__home_impr',
'initial_list_status:f',
'initial_list_status:w',
'term:36',
'term:60',
'emp_length:0',
'emp_length:1',
'emp_length:2-4',
'emp_length:5-6',
'emp_length:7-9',
'emp_length:10',
'mths_since_issue_d:<38',
'mths_since_issue_d:38-39',
'mths_since_issue_d:40-41',
'mths_since_issue_d:42-48',
'mths_since_issue_d:49-52',
'mths_since_issue_d:53-64',
'mths_since_issue_d:65-84',
'mths_since_issue_d:>84',
'int_rate:<9.548',
'int_rate:9.548-12.025',
'int_rate:12.025-15.74',
'int_rate:15.74-20.281',
'int_rate:>20.281',
'mths_since_earliest_cr_line:<140',
'mths_since_earliest_cr_line:141-164',
'mths_since_earliest_cr_line:165-247',
'mths_since_earliest_cr_line:248-270',
'mths_since_earliest_cr_line:271-352',
'mths_since_earliest_cr_line:>352',
'inq_last_6mths:0',
'inq_last_6mths:1-2',
'inq_last_6mths:3-6',
'inq_last_6mths:>6',
'acc_now_delinq:0',
'acc_now_delinq:>=1',
'dti:<=1.4',
'dti:1.4-3.5',
'dti:3.5-7.7',
'dti:7.7-10.5',
'dti:10.5-16.1',
'dti:16.1-20.3',
'dti:20.3-21.7',
'dti:21.7-22.4',
'dti:22.4-35',
'dti:>35',
'mths_since_last_delinq:Missing',
'mths_since_last_delinq:0-3',
'mths_since_last_delinq:4-30',
'mths_since_last_delinq:31-56',
'mths_since_last_delinq:>=57',
'mths_since_last_record:Missing',
'mths_since_last_record:0-2',
'mths_since_last_record:3-20',
'mths_since_last_record:21-31',
'mths_since_last_record:32-80',
'mths_since_last_record:81-86',
'mths_since_last_record:>86'
]]

In [ ]:
inputs_train_with_ref_cat.shape

In [ ]:
inputs_2015_with_ref_cat.shape

We have similar shapes, (89) differing only by the number of observations.

Now we want to calculate the PSI not only for the orignal indpt variables from the PD Model, but also for its  outcome: the credit score.

In [ ]:
df_scorecard = pd.read_csv('df_scorecard.csv', index_col = 0)

In [ ]:
df_scorecard

We need to add a column that will multiply by the intercept!

In [ ]:
inputs_train_with_ref_cat_w_intercept = inputs_train_with_ref_cat
inputs_train_with_ref_cat_w_intercept.insert(0, 'Intercept', 1)

We then order the columns of this data frame in the same way as the feature names from the scorecard 

In [ ]:
inputs_train_with_ref_cat_w_intercept = inputs_train_with_ref_cat_w_intercept[df_scorecard['Feature name'].values]
inputs_train_with_ref_cat_w_intercept.head()

In [ ]:
inputs_2015_with_ref_cat_w_intercept = inputs_2015_with_ref_cat
inputs_2015_with_ref_cat_w_intercept.insert(0, 'Intercept', 1)

inputs_2015_with_ref_cat_w_intercept = inputs_2015_with_ref_cat_w_intercept[df_scorecard['Feature name'].values]
inputs_2015_with_ref_cat_w_intercept.head()

Now we have to take only the scores and ensure that their shape is (90, 1) to make the dot product caluclation possible.

In [ ]:
scorecard_scores = df_scorecard['Score - Final']
scorecard_scores = scorecard_scores.values.reshape(90, 1)

In [ ]:
y_scores_train = inputs_train_with_ref_cat_w_intercept.dot(scorecard_scores)
y_scores_train.head()

This contains the scores of the observations from the original train data.

In [ ]:
y_scores_2015 = inputs_2015_with_ref_cat_w_intercept.dot(scorecard_scores)
y_scores_2015.head()

This contains the scores of the observations from the new data.

Let's combine each of these to their respective dataframes.

In [ ]:
inputs_train_with_ref_cat_w_intercept = pd.concat([inputs_train_with_ref_cat_w_intercept, y_scores_train], axis=1)
inputs_2015_with_ref_cat_w_intercept = pd.concat([inputs_2015_with_ref_cat_w_intercept, y_scores_2015], axis=1)

In [ ]:
inputs_train_with_ref_cat_w_intercept.columns.values[inputs_train_with_ref_cat_w_intercept.shape[1]-1] = 'Score'
inputs_2015_with_ref_cat_w_intercept.columns.values[inputs_2015_with_ref_cat_w_intercept.shape[1]-1] = 'Score'

In [ ]:
inputs_2015_with_ref_cat_w_intercept.head()

We calculate PSI with categorical dummy variables. This means we have to turn Score into dummy variables as well. We can create dummy variable intervals of 50 scores.

In [ ]:
inputs_train_with_ref_cat_w_intercept['Score:300-350'] = np.where((inputs_train_with_ref_cat_w_intercept['Score'] >= 300) & (inputs_train_with_ref_cat_w_intercept['Score'] < 350), 1, 0)
inputs_train_with_ref_cat_w_intercept['Score:350-400'] = np.where((inputs_train_with_ref_cat_w_intercept['Score'] >= 350) & (inputs_train_with_ref_cat_w_intercept['Score'] < 400), 1, 0)
inputs_train_with_ref_cat_w_intercept['Score:400-450'] = np.where((inputs_train_with_ref_cat_w_intercept['Score'] >= 400) & (inputs_train_with_ref_cat_w_intercept['Score'] < 450), 1, 0)
inputs_train_with_ref_cat_w_intercept['Score:450-500'] = np.where((inputs_train_with_ref_cat_w_intercept['Score'] >= 450) & (inputs_train_with_ref_cat_w_intercept['Score'] < 500), 1, 0)
inputs_train_with_ref_cat_w_intercept['Score:500-550'] = np.where((inputs_train_with_ref_cat_w_intercept['Score'] >= 500) & (inputs_train_with_ref_cat_w_intercept['Score'] < 550), 1, 0)
inputs_train_with_ref_cat_w_intercept['Score:550-600'] = np.where((inputs_train_with_ref_cat_w_intercept['Score'] >= 550) & (inputs_train_with_ref_cat_w_intercept['Score'] < 600), 1, 0)
inputs_train_with_ref_cat_w_intercept['Score:600-650'] = np.where((inputs_train_with_ref_cat_w_intercept['Score'] >= 600) & (inputs_train_with_ref_cat_w_intercept['Score'] < 650), 1, 0)
inputs_train_with_ref_cat_w_intercept['Score:650-700'] = np.where((inputs_train_with_ref_cat_w_intercept['Score'] >= 650) & (inputs_train_with_ref_cat_w_intercept['Score'] < 700), 1, 0)
inputs_train_with_ref_cat_w_intercept['Score:700-750'] = np.where((inputs_train_with_ref_cat_w_intercept['Score'] >= 700) & (inputs_train_with_ref_cat_w_intercept['Score'] < 750), 1, 0)
inputs_train_with_ref_cat_w_intercept['Score:750-800'] = np.where((inputs_train_with_ref_cat_w_intercept['Score'] >= 750) & (inputs_train_with_ref_cat_w_intercept['Score'] < 800), 1, 0)
inputs_train_with_ref_cat_w_intercept['Score:800-850'] = np.where((inputs_train_with_ref_cat_w_intercept['Score'] >= 800) & (inputs_train_with_ref_cat_w_intercept['Score'] <= 850), 1, 0)
# We create dummy variables for score intervals in the dataframe with old ("expected").

In [ ]:
inputs_2015_with_ref_cat_w_intercept['Score:300-350'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score'] >= 300) & (inputs_2015_with_ref_cat_w_intercept['Score'] < 350), 1, 0)
inputs_2015_with_ref_cat_w_intercept['Score:350-400'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score'] >= 350) & (inputs_2015_with_ref_cat_w_intercept['Score'] < 400), 1, 0)
inputs_2015_with_ref_cat_w_intercept['Score:400-450'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score'] >= 400) & (inputs_2015_with_ref_cat_w_intercept['Score'] < 450), 1, 0)
inputs_2015_with_ref_cat_w_intercept['Score:450-500'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score'] >= 450) & (inputs_2015_with_ref_cat_w_intercept['Score'] < 500), 1, 0)
inputs_2015_with_ref_cat_w_intercept['Score:500-550'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score'] >= 500) & (inputs_2015_with_ref_cat_w_intercept['Score'] < 550), 1, 0)
inputs_2015_with_ref_cat_w_intercept['Score:550-600'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score'] >= 550) & (inputs_2015_with_ref_cat_w_intercept['Score'] < 600), 1, 0)
inputs_2015_with_ref_cat_w_intercept['Score:600-650'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score'] >= 600) & (inputs_2015_with_ref_cat_w_intercept['Score'] < 650), 1, 0)
inputs_2015_with_ref_cat_w_intercept['Score:650-700'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score'] >= 650) & (inputs_2015_with_ref_cat_w_intercept['Score'] < 700), 1, 0)
inputs_2015_with_ref_cat_w_intercept['Score:700-750'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score'] >= 700) & (inputs_2015_with_ref_cat_w_intercept['Score'] < 750), 1, 0)
inputs_2015_with_ref_cat_w_intercept['Score:750-800'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score'] >= 750) & (inputs_2015_with_ref_cat_w_intercept['Score'] < 800), 1, 0)
inputs_2015_with_ref_cat_w_intercept['Score:800-850'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score'] >= 800) & (inputs_2015_with_ref_cat_w_intercept['Score'] <= 850), 1, 0)
# We create dummy variables for score intervals in the dataframe with new ("actual").

Now we are finally ready to calculate PSI!

### Population Stability Index: Calculation and Interpretation

To calculate PSI for a single original feature, we need the proportions of observations per category for all of its dummy variable categories. But how do we do this simultaneously for all variables?

In [ ]:
PSI_calc_train = inputs_train_with_ref_cat_w_intercept.sum() / inputs_train_with_ref_cat_w_intercept.shape[0]

In [ ]:
PSI_calc_2015 = inputs_2015_with_ref_cat_w_intercept.sum() / inputs_2015_with_ref_cat_w_intercept.shape[0]

In [ ]:
PSI_calc = pd.concat([PSI_calc_train, PSI_calc_2015], axis=1)

In [ ]:
PSI_calc = PSI_calc.reset_index()
PSI_calc['Original_feature_name'] = PSI_calc['index'].str.split(':').str[0]

^This cell makes the last col of the dataframe to contain the name of the original indpt variable.

In [ ]:
PSI_calc.columns = ['index', 'Proportions_Train', 'Proportions_New', 'Original_feature_name']

In [ ]:
PSI_calc = PSI_calc[np.array(['index', 'Original_feature_name', 'Proportions_Train', 'Proportions_New'])]

In [ ]:
PSI_calc['Original_feature_name']

We have two redundant rows. We don't need the intercept and score rows.

In [ ]:
PSI_calc= PSI_calc[(PSI_calc['index'] != 'Intercept') & (PSI_calc['index'] != 'Score')]

In [ ]:
PSI_calc['Contribution'] = np.where((PSI_calc['Proportions_Train'] == 0) | (PSI_calc['Proportions_New'] == 0), 0, 
                                    (PSI_calc['Proportions_New'] - PSI_calc['Proportions_Train']) * np.log(PSI_calc['Proportions_New'] / PSI_calc['Proportions_Train']))


^This cell sets contribution to 0 if either of the proportions are equal to 0

In [ ]:
PSI_calc

Now we have to sum the contributions by original features. We can do this for all features by grouping the PSI_calc dataframe by original feature name, taking the contribution column, and applying sum to it.

In [ ]:
PSI_calc.groupby('Original_feature_name')['Contribution'].sum()

To interpret these values, we refer to the PSI population difference chart.

From acc_now_delinq to home_ownership, the values are less than 0.1 --> Little to no difference in population.

The first feature that appears to be problematic is initial_list_status. It is much larger than .25 (meaning there is a big difference in people applying for loans now than the people applying for loans in previous years).

The next 3 features have little to no difference in population, so no concerns here.

mths_since_issue_d shows absolute difference in population.

The rests of the values are low as well, so there are no concerns there.

The most serious change we observe is that the populations differ with respect ot credit scores. This implies our model outputs are much different than those before. So, we must train the whole model again by including the new population in the training data set.